In [1]:
import pandas as pd
from pandas.io.json import json_normalize 
import urllib.request
import json
import tldextract

In [2]:
def clear_url(origin_url):
    return tldextract.extract(origin_url).registered_domain

### IAB vendor list

In [3]:
with urllib.request.urlopen('https://vendorlist.consensu.org/vendorlist.json') as response:
   vendorlist = json.loads(response.read().decode())

df_iab = json_normalize(vendorlist['vendors'])
df_iab['policyUrl'] = df_iab['policyUrl'].apply(clear_url)
df_iab.head()

,featureIds,id,legIntPurposeIds,name,policyUrl,purposeIds
0,"[1, 2]",8,"[3, 5]",Emerse Sverige AB,emerse.com,"[1, 2, 4]"
1,[3],12,[],BeeswaxIO Corporation,beeswax.com,"[1, 3, 5]"
2,[3],28,[],"TripleLift, Inc.",triplelift.com,"[1, 3]"
3,"[1, 2, 3]",9,[],AdMaxim Inc.,admaxim.com,"[1, 2, 3, 4, 5]"
4,[],27,[],ADventori SAS,adventori.com,"[1, 3]"


### Google providers

In [4]:
df_google = pd.read_csv('https://storage.googleapis.com/adx-rtb-dictionaries/providers.csv')
df_google['policy_url'] = df_google['policy_url'].apply(clear_url)
df_google.head()

,provider_id,provider_name,policy_url,domains
0,3,2KDirect Inc.,ipromote.com,ad1.ipromote.com cdn.ipromote.com fra*.ipromot...
1,7,Amazon,amazon.co.uk,*eu-central-1.amazonaws.com *eu-west-1.amazona...
2,11,Acuityads,acuityads.com,acuityplatform.com cdn.acuityplatform.com clic...
3,12,LiveRamp,liveramp.com,idsync.rlcdn.com pippio.com rc.rlcdn.com rlcdn...
4,15,Adara Media,adara.com,adaraanalytics.com ag.yieldoptimizer.com cdnfs...


### Intersection

In [5]:
intersected_df = pd.merge(df_iab, df_google, how='left', left_on='policyUrl', right_on='policy_url')
clean_df = intersected_df[['id','name','policyUrl','provider_id','provider_name']]
clean_df.head()

,id,name,policyUrl,provider_id,provider_name
0,8,Emerse Sverige AB,emerse.com,797.0,Emerse
1,12,BeeswaxIO Corporation,beeswax.com,1858.0,Beeswax
2,28,"TripleLift, Inc.",triplelift.com,933.0,TripleLift
3,9,AdMaxim Inc.,admaxim.com,827.0,AdMaxim
4,27,ADventori SAS,adventori.com,1248.0,Adventori
